In [2]:
import numpy as np
import pandas as pd
import re
import glob
import os 
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.linear_model import LogisticRegression

In [ ]:
w_stack_train=pd.read_csv("..//final_scores/w_train.csv")
w_stack_test=pd.read_csv("..//final_scores/w_test.csv")
nw_stack_train=pd.read_csv("..//final_scores/nw_train.csv")
nw_stack_test=pd.read_csv("..//final_scores/nw_test.csv")

In [4]:
#meta features

In [ ]:
mt_train=pd.read_csv("..//l2_meta_features/meta_train_l1l2.csv")
mt_test=pd.read_csv("..//l2_meta_features/meta_test_l1l2.csv")
mt_train_a=pd.read_csv("..//l2_meta_features/meta_train_l2.csv")
mt_test_a=pd.read_csv("..//l2_meta_features/meta_test_l2.csv")

In [ ]:
#merge all 

In [ ]:
stack_train=w_stack_train.merge(nw_stack_train,on="ID").merge(mt_train,
        on="ID").merge(mt_train_a,on="ID")
stack_test=w_stack_test.merge(nw_stack_test,on="ID").merge(mt_test,
        on="ID").merge(mt_test_a,on="ID")

In [ ]:
y=pd.read_csv("..//bases/training_variants").Class.values - 1
X_train=np.array(stack_train.drop("ID",axis=1))
X_test=np.array(stack_test.drop("ID",axis=1))

In [5]:
#Ridge vs Lasso searching
param_test= {
    "C" : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "penalty": ["l1","l2"]
}

In [ ]:
gsearch=GridSearchCV(estimator=LogisticRegression(), param_grid = param_test,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
gsearch.fit(X_train,y)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_


In [6]:
# best model to complete

In [ ]:
clf = LogisticRegression()                     
clf.fit(X_train,y)
y_test=clf.predict_proba(X_test)

classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
sub = pd.DataFrame(y_test, columns=classes)
sub['ID'] = ID_test
sub.to_csv("scores/stacking_final.csv",index=False)

In [10]:
# which model impacts more ? 

In [ ]:
features=np.asarray(stack_test.drop("ID",axis=1).columns)
imp=clf.feature_importances_
feat_imp=pd.DataFrame(imp,features,columns=["feat_imp"])
fi=feat_imp.sort_values(by="feat_imp",ascending=False)